In [130]:
import requests
from bs4 import BeautifulSoup
community_page = requests.get('https://www.ravensoftware.com/community/')
community_soup = BeautifulSoup(community_page.text, 'html.parser')

In [131]:
pfc = community_soup.find_all(class_='post-feature-container')
pc = community_soup.find_all(class_='post-container')
posts = pfc + pc

In [132]:
from datetime import datetime
import textwrap
import re
RAVEN_URI='https://www.ravensoftware.com'
DISCORD_CHARACTERS_LIMIT = 2000

def get_post_updated_date(date_div):
    if date_div is not None:
        date_text = date_div.text.strip()
        if date_text == '':
            return None
        return datetime.strptime(date_text, '%b %d, %Y')
    return None

def format_content_url(text=''):
    if text.startswith('/community') or text.startswith('/content'):
        return RAVEN_URI + text
    return text

def get_post_content(content_url=''):
    if not content_url:
        return ''
    content_page = requests.get(content_url)
    content_soup = BeautifulSoup(content_page.text, 'html.parser')
    return content_soup

def read_blog_article(content):
    #TODO write function logic
    return

def get_releases_sections(content):
    releases_sections = content.find_all('h2')
    releases_sections = list(map(lambda x: x.get_text(), releases_sections))
    return releases_sections

def div_contains_section(div, sections):
    for section in sections:
        if section in div.get_text():
            return section
    return False

def split_text_limit_characters(text, limit=DISCORD_CHARACTERS_LIMIT):
    text = re.sub(r'\n\n+', '\n', text).strip()
    return textwrap.wrap(text,
                        width=limit,
                        break_long_words=False,
                        replace_whitespace=False
                        )

def read_patch_notes(content):
    notes_object = {}
    releases_sections = get_releases_sections(content)
    main_div = content.find('div', class_='aem-Grid aem-Grid--12 aem-Grid--default--12')
    actual_section = None
    for div in main_div.select('div[class*="aem-GridColumn aem-GridColumn--default--12"]'):
        is_section = div_contains_section(div, releases_sections)
        if not is_section and not actual_section:
            continue
        if is_section:
            actual_section = is_section
            notes_object[actual_section] = {'text': '', 'images': []}
        notes_object[actual_section]['text'] = notes_object[actual_section]['text'] + div.get_text()

        img_tags = div.find_all('img')
        for img in img_tags:
            if 'src' not in img.attrs:
                continue
            src = img.attrs['src']
            if 'horizontal-long-line.png' in src:
                continue
            notes_object[actual_section]['images'].append(format_content_url(src))
    return notes_object

for post in posts:
    updated_date = get_post_updated_date(post.find(class_='post-feature-date') or post.find(class_='post-date'))
    if updated_date is None:
        continue
    if updated_date > datetime.fromisoformat('2022-04-05'):
        content = get_post_content(format_content_url(post.find('a').attrs['href']))
        if not content:
            continue
        if content.find(class_='blog-body'):
            patch_object = read_patch_notes(content.find(class_='blog-body'))

        if content.find(class_='blog-body-container'):
            read_blog_article(content.find(class_='body-content'))


In [133]:
from IPython.display import display, Image
from IPython.core.display import HTML
# for img in patch_object['SEASON THREE LAUNCH']['images']:
#     display(Image(url=img, width=500, height=500))
# with open('result.txt', 'a') as fp:
#     fp.write(patch_object['SEASON THREE LAUNCH']['text'])

import textwrap
import re
text =patch_object['SEASON THREE LAUNCH']['text']
text = re.sub(r'\n\n+', '\n', text).strip()
wrapped = textwrap.wrap(text,
                        width=2000,
                        break_long_words=False,
                        replace_whitespace=False
                        )
for part in wrapped:
    print(part)
    with open('result.txt', 'a') as fp:
        fp.write(part)
    print('\n\n\n\n----------')


KeyError: 'SEASON THREE LAUNCH'